<a href="https://colab.research.google.com/github/amun83/Koscom-Financial-Data-Bootcamp/blob/master/FD104_Text_Mining_OilPrice_com_News_Articles_Step_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Scraping News Articles: https://oilprice.com/

In [0]:
ON_COLAB  = True
DATA_PATH = ''

In [0]:
!pip install -q --upgrade pandas_datareader
!pip install -q --upgrade finance-datareader
!pip install -q --upgrade html5lib
!pip install -q --upgrade lxml
!pip install -q --upgrade seaborn
!pip install -q --upgrade gspread

In [0]:
# utilities
import datetime
import re
import csv
from IPython.display import display

# web scraping
import requests
import urllib.parse
from bs4 import BeautifulSoup as bs
import pandas_datareader as pdr
import FinanceDataReader as fdr

# Math/Stat
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
#import math

# Machine Learning
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM

# visualization
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
if ON_COLAB :
  from google.colab import drive
  drive.mount('/gdrive')
  DATA_PATH = '/gdrive/My Drive/sandbox/'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
pattern = re.compile(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s(\d\d),\s(\d{4})')
# Example: By Julianne Geiger - Dec 14, 2018, 8:00 PM CST

In [0]:
def get_news_urls_by_page(page_num):
  try:
    get_news_urls_by_page.counter += 1
  except:
    get_news_urls_by_page.counter = 0
    
  page_url = 'https://oilprice.com/Latest-Energy-News/World-News/Page-' + str(page_num) + '.html'
  # https://oilprice.com/Latest-Energy-News/World-News/Page-865.html

  try:
    r = requests.get(page_url)
    if get_news_urls_by_page.counter % 10 == 0:
      print('Request(GET) >> OK: ' + page_url)
  except:
    print('Request(GET) >> Failed: ' + page_url)
    return None

  soup = bs(r.text)
  items = soup.find_all('div', {'class':'categoryArticle__content'})

  if not items:
    print('Request(GET) >> Empty Page: ' + page_url)
    return None
  
  return [item.find('a', href = True)['href'] for item in items]

In [0]:
def get_news_content(url):
  try:
    get_news_content.counter += 1
  except:
    get_news_content.counter = 0
  
  try:
    r = requests.get(url)
    if get_news_content.counter % 100 == 0:
      print('[' + str(get_news_content.counter) + '] Request(GET) >> OK: ' + url)
  except:
    print('[' + str(get_news_content.counter) + '] Request(GET) >> Failed: ' + url)
    return None

  soup = bs(r.text)

  title = soup.find('h1').text

  date = soup.find('span', {'class':'article_byline'}).text

  match = re.search(pattern, date)
  mmm, dd, yyyy = match.groups(0)
  date = yyyy + "-" + mmm + "-" + dd

  raw_content = soup.find('div', {'id':'news-content'}).find_all('p')
  content = ' '.join([p.text.strip() for p in raw_content])
  
  author = soup.find('p', {'class':'authorBox__authorName'}).text
  
  return date, title, content, author

In [33]:
news_urls =[]

for i in range(1, 3):
  urls = get_news_urls_by_page(i)
  if not urls:
    break
    
  news_urls += urls

Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-1.html


In [34]:
print(len(news_urls))
print(news_urls[-1])

20
https://oilprice.com/Latest-Energy-News/World-News/Climate-Passport-For-People-Driven-Away-By-Global-Warming-Proposed.html


In [0]:
with open(DATA_PATH + 'oilprice.com_news_urls.csv', 'w') as f:
  fw = csv.writer(f)
  fw.writerows([[url] for url in news_urls])

In [36]:
news = [get_news_content(url) + (url.split('/')[-1],) for url in news_urls]

[0] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Shell-in-Talks-To-Buy-Endeavor-Energy.html


In [37]:
print(len(news))
display(news[-5:-1])

20


[('2018-Dec-13',
  'Tariffs Hit U.S. Solar Market In Q3, But Plans Show Strong Rebound',
  'The U.S. tariffs on imported solar cells and modules created uncertainty in the market earlier this year, resulting in quarterly additions of utility-scale solar dropping in Q3 below 1 gigawatt for the first time since 2015, yet the project pipeline points to a strong rebound in Q4, a new report by Wood Mackenzie Power & Renewables and the Solar Energy Industries Association (SEIA) showed on Thursday. The tariffs, enacted in early February, created uncertainty in the market in the latter half of 2017 and in early 2018, pushing back projects until operators had a clearer view about the implementation of the tariffs and their impact on project economics, according to the Q3 U.S. Solar Market Insight Report. While utility-scale solar installations dipped in the third quarter, the residential solar market continued to stabilize this year after a weak 2017. The overall U.S. market installed 1.7 GW of

In [0]:
with open(DATA_PATH + 'oilprice.com_news.csv', 'w') as f:
  fw = csv.writer(f)
  fw.writerows(news)